In [3]:
import pandas as pd
from plotly import express as px
import numpy as np

In [35]:
df.loc[df['policy'] == 'mlp','reward_calculator_version']

306    19
307    19
308    19
309    19
310    19
       ..
420    20
421    20
422    20
423    20
424    20
Name: reward_calculator_version, Length: 119, dtype: int64

In [39]:
for additional_percentage in [0.2, 1]:
    df = pd.read_csv(f'/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments_____/processed_tesnorboard/duration_gap.csv')
    df = df[df['additional_percentage'] == additional_percentage]
    df.loc[df['policy'] == 'mlp','policy'] = df.loc[df['policy'] == 'mlp','reward_calculator_version'].apply(lambda x: f"{x}_agent")
    df = df.drop(columns=['run_name']).groupby('policy').mean().reset_index()
    df[['event_code', 'is_triggered']] = np.zeros((len(df), 2))
    df[['event_code', 'is_triggered']] = df['policy'].str.split('_', expand=True)
    bar_fig = px.bar(df, x='policy', y='value', title=f'Duration gap by policy (additional percentage: {additional_percentage})', color='event_code', color_discrete_sequence=px.colors.qualitative.Alphabet)
    bar_fig.show()
    bar_fig = px.bar(df, x='policy', y='value', title='Duration gap by policy', color='is_triggered', color_discrete_sequence=px.colors.qualitative.Alphabet)
    bar_fig.show()


In [8]:
for additional_percentage in [0.2, 1]:
    df = pd.read_csv('/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments_____/processed_tesnorboard/p_values.csv')
    df = df[df['additional_percentage'] == additional_percentage]
    df = df.drop(columns=['run_name']).groupby('policy').mean().reset_index()
    df[['event_code', 'is_triggered']] = df['policy'].str.split('_', expand=True)
    bar_fig = px.bar(df, x='policy', y='value', title=f'P values by policy (additional percentage: {additional_percentage})', color='event_code', color_discrete_sequence=px.colors.qualitative.Alphabet)
    bar_fig.show()
    bar_fig = px.bar(df, x='policy', y='value', title='P values by policy', color='is_triggered', color_discrete_sequence=px.colors.qualitative.Alphabet)
    bar_fig.show()

In [60]:
import os
import splunklib.client as client
import splunklib.results as results
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import psutil

def monitor_search_resources(host, port, username, password, search_query, duration_minutes):
    # Connect to Splunk
    service = client.connect(
        host=host,
        port=port,
        username=username,
        password=password
    )

    # Start the search job
    job = service.jobs.create(search_query)

    start_time = datetime.now()
    end_time = start_time + timedelta(minutes=duration_minutes)

    while datetime.now() < end_time:
        # Get the latest statistics
        job.refresh()
        stats = job.content
        
        # Extract relevant resource metrics
        scan_count = stats.get('scanCount', 0)
        event_count = stats.get('eventCount', 0)
        result_count = stats.get('resultCount', 0)
        disk_usage = stats.get('diskUsage', 0)
        run_duration = stats.get('runDuration', 0)
        pid = int(stats.get('pid', 0))
        print('pid: ', pid)
        if pid != 0:
            try:
                process = psutil.Process(pid)
                with process.oneshot():
                    cpu_percent = process.cpu_percent(interval=1)
                    cpu_num = psutil.cpu_count()
                    cpu_times = process.cpu_times()
                    memory_info = process.memory_info()
                    io_counters = process.io_counters()
            except psutil.NoSuchProcess:
                print(f"Process with PID {pid} not found.")
                return None
            except psutil.AccessDenied:
                print(f"Access denied to process with PID {pid}.")
                return None
        else:
            cpu_percent = 0
            cpu_num = 0
            cpu_times = 0
            memory_info = 0
            io_counters = 0

        
        # Print or store the metrics as needed
        print(f"Timestamp: {datetime.now()}")
        print(f"Scan Count: {scan_count}")
        print(f"Event Count: {event_count}")
        print(f"Result Count: {result_count}")
        print(f"Disk Usage: {disk_usage} bytes")
        print(f"Run Duration: {run_duration} seconds")
        print(f"CPU Usage: {cpu_percent} %")
        print(f"CPU Num: {cpu_num}")
        print(f"CPU Times: {cpu_times}")
        print(f"Memory Info: {memory_info}")
        print(f"IO Counters: {io_counters}")
        # print(f"Network In: {network_bytes_in} bytes")
        # print(f"Network Out: {network_bytes_out} bytes")
        print("---")
        
        time.sleep(0.01)

    # Cancel the job when done
    job.cancel()

# Example usage
if __name__ == "__main__":
    load_dotenv('/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/src/.env')
    monitor_search_resources(
        host=os.getenv("SPLUNK_HOST"),
        port=os.getenv("SPLUNK_PORT"),
        username=os.getenv("SPLUNK_USERNAME"),
        password=os.getenv("SPLUNK_PASSWORD"),
        search_query='search `wineventlog_security` EventCode=5140 ShareName IN("\\\\*\\ADMIN$","\\\\*\\C$","*\\\\*\\IPC$") AccessMask= 0x1  earliest=1717437600.0 latest=1717452000.0| stats min(_time) as firstTime max(_time) as lastTime count by ShareName SourceAddress ObjectType AccountName AccountDomain SourcePort AccessMask Computer | `security_content_ctime(firstTime)` | `security_content_ctime(lastTime)` | `network_share_discovery_via_dir_command_filter`',
        # search_query='search `wineventlog_system` EventCode=7040  Message = "*service was changed from demand start to disabled."  earliest=1717408800.0 latest=1717416000.0| stats count min(_time) as firstTime max(_time) as lastTime by ComputerName EventCode Message User Sid service service_name | `security_content_ctime(firstTime)` | `security_content_ctime(lastTime)` | `windows_event_for_service_disabled_filter`',
        duration_minutes=3
    )

pid:  0
Timestamp: 2024-10-08 12:40:27.444310
Scan Count: 0
Event Count: 0
Result Count: 0
Disk Usage: 28672 bytes
Run Duration: 0 seconds
CPU Usage: 0 %
CPU Num: 0
CPU Times: 0
Memory Info: 0
IO Counters: 0
---
pid:  0
Timestamp: 2024-10-08 12:40:27.464351
Scan Count: 0
Event Count: 0
Result Count: 0
Disk Usage: 28672 bytes
Run Duration: 0 seconds
CPU Usage: 0 %
CPU Num: 0
CPU Times: 0
Memory Info: 0
IO Counters: 0
---
pid:  0
Timestamp: 2024-10-08 12:40:27.486769
Scan Count: 0
Event Count: 0
Result Count: 0
Disk Usage: 28672 bytes
Run Duration: 0 seconds
CPU Usage: 0 %
CPU Num: 0
CPU Times: 0
Memory Info: 0
IO Counters: 0
---
pid:  0
Timestamp: 2024-10-08 12:40:27.506717
Scan Count: 0
Event Count: 0
Result Count: 0
Disk Usage: 36864 bytes
Run Duration: 0 seconds
CPU Usage: 0 %
CPU Num: 0
CPU Times: 0
Memory Info: 0
IO Counters: 0
---
pid:  28617
Access denied to process with PID 28617.
